In [3]:
from utils.hole_tracker import HoleTracker

tracker = HoleTracker(10, 10, 10)

tracker._add_p_detection(100, [1]*3)
tracker._add_p_detection(100, [1]*2)


TypeError: found wrong length in new detection point data to be added! got input length = 2 but require length = 3